In [1]:
import numpy as np
import cv2
from cv2 import dnn

In [7]:
net = cv2.dnn.readNetFromCaffe('.\\model\\colorization_deploy_v2.prototxt', '.\\model\\colorization_release_v2.caffemodel')
points = np.load('.\\model\\pts_in_hull.npy')
img_path = ('.\\images\\sample\\sample10.jpg')
class8= net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
points = points.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [points.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

img = cv2.imread(img_path)
scaled = img.astype("float32") / 255.0
lab_img = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

resized_img = cv2.resize(lab_img, (224, 224))
# split the L channel
L = cv2.split(resized_img)[0]
# mean subtraction
L -= 50

net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
# resize the predicted 'ab' volume to the same dimensions as our
# input image
ab = cv2.resize(ab, (img.shape[1], img.shape[0]))


# Take the L channel from the image
L = cv2.split(lab_img)[0]
# Join the L channel with predicted ab channel
colorized_img = np.concatenate((L[:, :, np.newaxis], ab), axis=2)

# Then convert the image from Lab to BGR 
colorized_img = cv2.cvtColor(colorized_img, cv2.COLOR_LAB2BGR)
colorized_img = np.clip(colorized_img, 0, 1)

# change the image to 0-255 range and convert it from float32 to int
colorized_img = (255 * colorized_img).astype("uint8")
    # Let's resize the images and show them together
img = cv2.resize(img,(640,640))
colorized = cv2.resize(colorized_img,(640,640))
 
result = cv2.hconcat([img,colorized])
 
cv2.imshow("Grayscale -> Colour", result)
 
cv2.waitKey(0)


-1